In [1]:
! python --version

Python 3.12.0


In [2]:
%%capture
! pip install llama-index
! pip install psycopg2-binary
! pip install SQLAlchemy
! pip install python-dotenv
! pip install llama-index-llms-bedrock
! pip install llama-index-embeddings-bedrock

In [3]:
from sqlalchemy import (
    create_engine,
)
import os

from dotenv import load_dotenv

load_dotenv(verbose=True, dotenv_path="../../../.env")

engine = create_engine(f"postgresql+psycopg2://postgres:{
                       os.environ["PG_VECTOR_PW"]}@localhost:5432/{os.environ["PG_VECTOR_DB"]}")

In [4]:
# Testing our connection
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM people")
    print(cursor.fetchall())
    
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM planet")
    print(cursor.fetchall())

[(1, 'Luke Skywalker', 172, 77.0, 'blond', 'fair', 'blue', '19BBY', 'male', 1, datetime.datetime(2014, 12, 9, 13, 50, 51, 644000), datetime.datetime(2014, 12, 20, 21, 17, 56, 891000), 'https://swapi.co/api/people/1/'), (2, 'C-3PO', 167, 75.0, 'n/a', 'gold', 'yellow', '112BBY', 'n/a', 1, datetime.datetime(2014, 12, 10, 15, 10, 51, 357000), datetime.datetime(2014, 12, 20, 21, 17, 50, 309000), 'https://swapi.co/api/people/2/'), (3, 'R2-D2', 96, 32.0, 'n/a', 'white, blue', 'red', '33BBY', 'n/a', 8, datetime.datetime(2014, 12, 10, 15, 11, 50, 376000), datetime.datetime(2014, 12, 20, 21, 17, 50, 311000), 'https://swapi.co/api/people/3/'), (4, 'Darth Vader', 202, 136.0, 'none', 'white', 'yellow', '41.9BBY', 'male', 1, datetime.datetime(2014, 12, 10, 15, 18, 20, 704000), datetime.datetime(2014, 12, 20, 21, 17, 50, 313000), 'https://swapi.co/api/people/4/'), (5, 'Leia Organa', 150, 49.0, 'brown', 'light', 'brown', '19BBY', 'female', 2, datetime.datetime(2014, 12, 10, 15, 20, 9, 791000), datetim

In [5]:
from llama_index.core import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding

Settings.llm = Bedrock(
    # model="anthropic.claude-3-haiku-20240307-v1:0",
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=os.environ["AWS_SESSION_TOKEN"],
    region_name=os.environ["AWS_DEFAULT_REGION"],
)

Settings.embed_model = BedrockEmbedding(
        model_name="cohere.embed-english-v3",
        region_name=os.environ["AWS_DEFAULT_REGION"],
    )

In [6]:
from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine

all_table_names = ["people", "planet"]

sql_database = SQLDatabase(engine, include_tables=all_table_names)

In [7]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display

In [8]:
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = []
for table_name in all_table_names:
    table_schema_objs.append(SQLTableSchema(table_name=table_name))
    
print(table_schema_objs)

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=2)
)

def format_sql_results_as_markdown_table(sql_results, headers):
    """
    Formats the SQL results as a markdown table
    """
    if not sql_results and not headers:
        return "No results found."
    
    header_row = "| " + " | ".join(headers) + " |"
    separator_row = "| " + " | ".join(["---"] * len(headers)) + " |"

    table_rows = []
    for row in sql_results:
        table_row = "|"
        for j, _ in enumerate(headers):
            table_row += f" {str(row[j])} |"
        table_rows.append(table_row)

    markdown_table = "\n".join([header_row, separator_row] + table_rows)
    return markdown_table

response_template = """
### Question

**{number}. {question}**

### Answer

{response}

### Generated SQL Query
```sql
{sql}
```

### SQL Results

{sql_results}

"""

def text_to_sql(query_engine, question, number=1):
    """
    Calls the query engine with the given question and displays the response as a markdown cell
    """
    engine_response = query_engine.query(question)
    display(Markdown(response_template.format(
            number=number,
            question=question,
            response=str(engine_response),
            sql=engine_response.metadata["sql_query"],
            sql_results=format_sql_results_as_markdown_table(engine_response.metadata["result"], engine_response.metadata["col_keys"]),
        )))

[SQLTableSchema(table_name='people', context_str=None), SQLTableSchema(table_name='planet', context_str=None)]


In [9]:
text_to_sql(query_engine, "How many tables are there in the database?", number=1)


### Question

**1. How many tables are there in the database?**

### Answer

Based on the SQL query and response, there are 2 tables in the database schema 'public'.

### Generated SQL Query
```sql
SELECT COUNT(*) AS num_tables FROM information_schema.tables WHERE table_schema = 'public';
```

### SQL Results

| num_tables |
| --- |
| 2 |



In [10]:
text_to_sql(query_engine, "How many of the planets are rocky?", number=2)


### Question

**2. How many of the planets are rocky?**

### Answer

According to the SQL query and its result, there are 27 planets in the database that have a rocky, mountainous, or desert terrain. Therefore, the response to the input question "How many of the planets are rocky?" is:

There are 27 rocky planets.

### Generated SQL Query
```sql
SELECT COUNT(*) AS rocky_planet_count
FROM planet
WHERE terrain LIKE '%rocky%' OR terrain LIKE '%mountain%' OR terrain LIKE '%desert%';
```

### SQL Results

| rocky_planet_count |
| --- |
| 27 |



In [11]:
text_to_sql(query_engine, "What are the names, eye color and hair color of people who have blue eyes? Arrange the names in reverse alphabetical order.", number=3)


### Question

**3. What are the names, eye color and hair color of people who have blue eyes? Arrange the names in reverse alphabetical order.**

### Answer

Here are the names, eye colors, and hair colors of people who have blue eyes, arranged in reverse alphabetical order:

Wilhuff Tarkin - Blue eyes, auburn/grey hair
Tarfful - Blue eyes, brown hair  
Ric Olié - Blue eyes, brown hair
Qui-Gon Jinn - Blue eyes, brown hair
Owen Lars - Blue eyes, brown/grey hair
Mon Mothma - Blue eyes, auburn hair
Mas Amedda - Blue eyes, no hair
Luminara Unduli - Blue eyes, black hair
Luke Skywalker - Blue eyes, blond hair
Lobot - Blue eyes, no hair
Jocasta Nu - Blue eyes, white hair
Jek Tono Porkins - Blue eyes, brown hair
Finis Valorum - Blue eyes, blond hair
Cliegg Lars - Blue eyes, brown hair
Chewbacca - Blue eyes, brown hair
Beru Whitesun lars - Blue eyes, brown hair
Barriss Offee - Blue eyes, black hair
Anakin Skywalker - Blue eyes, blond hair
Adi Gallia - Blue eyes, no hair

### Generated SQL Query
```sql
SELECT name, eye_color, hair_color
FROM people
WHERE eye_color = 'blue'
ORDER BY name DESC;
```

### SQL Results

| name | eye_color | hair_color |
| --- | --- | --- |
| Wilhuff Tarkin | blue | auburn, grey |
| Tarfful | blue | brown |
| Ric Olié | blue | brown |
| Qui-Gon Jinn | blue | brown |
| Owen Lars | blue | brown, grey |
| Mon Mothma | blue | auburn |
| Mas Amedda | blue | none |
| Luminara Unduli | blue | black |
| Luke Skywalker | blue | blond |
| Lobot | blue | none |
| Jocasta Nu | blue | white |
| Jek Tono Porkins | blue | brown |
| Finis Valorum | blue | blond |
| Cliegg Lars | blue | brown |
| Chewbacca | blue | brown |
| Beru Whitesun lars | blue | brown |
| Barriss Offee | blue | black |
| Anakin Skywalker | blue | blond |
| Adi Gallia | blue | none |



In [12]:
text_to_sql(query_engine, "List the top 10 planets that have rocky characteristics, in alphabetical order?", number=4)


### Question

**4. List the top 10 planets that have rocky characteristics, in alphabetical order?**

### Answer

Based on the SQL query and results, here is a synthesized response to the input question "List the top 10 planets that have rocky characteristics, in alphabetical order?":

The top 10 planets with rocky characteristics, listed in alphabetical order, are:

1. Alderaan - Featuring grasslands and mountains
2. Bestine IV - Rocky islands and oceans
3. Cato Neimoidia - Mountains, fields, forests, and rock arches
4. Coruscant - A cityscape with mountains
5. Dantooine - Oceans, savannas, mountains, and grasslands
6. Endor - Forests, mountains, and lakes
7. Geonosis - Rock, desert, mountains, and barren terrain
8. Hoth - Tundra, ice caves, and mountain ranges
9. Iktotch - Rocky terrain
10. Iridonia - Rocky canyons and acid pools

These planets exhibit rocky features such as mountains, canyons, rock arches, and rocky islands or terrain, making them ideal candidates for the list of top rocky planets.

### Generated SQL Query
```sql
SELECT p.name, p.terrain
FROM planet p
WHERE p.terrain LIKE '%rocky%' OR p.terrain LIKE '%mountain%'
ORDER BY p.name
LIMIT 10;
```

### SQL Results

| name | terrain |
| --- | --- |
| Alderaan | grasslands, mountains |
| Bestine IV | rocky islands, oceans |
| Cato Neimoidia | mountains, fields, forests, rock arches |
| Coruscant | cityscape, mountains |
| Dantooine | oceans, savannas, mountains, grasslands |
| Endor | forests, mountains, lakes |
| Geonosis | rock, desert, mountain, barren |
| Hoth | tundra, ice caves, mountain ranges |
| Iktotch | rocky |
| Iridonia | rocky canyons, acid pools |



In [13]:
text_to_sql(query_engine, "In th top 10 planets that have rocky characteristics, who are the men and describe their skin color and eye color?", number=5)


### Question

**5. In th top 10 planets that have rocky characteristics, who are the men and describe their skin color and eye color?**

### Answer

Based on the query results for the top 10 rocky planets, the men found on those planets along with their skin and eye colors are:

1. Eeth Koth from Iridonia has brown skin and brown eyes.
2. Saesee Tiin from Iktotch has pale skin and orange eyes. 
3. Nien Nunb from Sullust has grey skin and black eyes.
4. Jek Tono Porkins from Bestine IV has fair skin and blue eyes.

The query filtered for planets with rocky terrain characteristics, joined with the people table to get the names and physical attributes of males from those planets, ordered by descending planet population, and limited the results to the top 10 records.

### Generated SQL Query
```sql
SELECT p.name, pe.name, pe.skin_color, pe.eye_color
FROM planet p
JOIN people pe ON p.id = pe.planet_id
WHERE p.terrain LIKE '%rocky%'
  AND pe.gender = 'male'
ORDER BY p.population DESC
LIMIT 10;
```

### SQL Results

| name | name | skin_color | eye_color |
| --- | --- | --- | --- |
| Iridonia | Eeth Koth | brown | brown |
| Iktotch | Saesee Tiin | pale | orange |
| Sullust | Nien Nunb | grey | black |
| Bestine IV | Jek Tono Porkins | fair | blue |



In [14]:
text_to_sql(query_engine, "What are the names and heights of all people who are from planets with a climate of 'temperate'?", number=6)


### Question

**6. What are the names and heights of all people who are from planets with a climate of 'temperate'?**

### Answer

Based on the SQL query and results, here is a response to the input question "What are the names and heights of all people who are from planets with a climate of 'temperate'?":

The query retrieves the name and height columns from the people table, joining with the planet table to filter for only those people whose planet has a 'temperate' climate. The results are ordered by height in descending order.

Some notable people from temperate planets include:

- Lama Su, with a height of 229 cm
- Roos Tarpals, 224 cm 
- Taun We, 213 cm
- Rugor Nass, 206 cm
- Ki-Adi-Mundi, 198 cm
- Jar Jar Binks, 196 cm
- Mas Amedda, 196 cm

The tallest is Lama Su at 229 cm, while some of the shorter ones include Leia Organa at 150 cm, Watto at 137 cm, and the droids R2-D2 at 96 cm and Wicket at 88 cm.

The results cover a diverse range of species and characters from various planets with temperate climates across the Star Wars universe.

### Generated SQL Query
```sql
SELECT people.name, people.height
FROM people
JOIN planet ON people.planet_id = planet.id
WHERE planet.climate = 'temperate'
ORDER BY people.height DESC;
```

### SQL Results

| name | height |
| --- | --- |
| Lama Su | 229 |
| Roos Tarpals | 224 |
| Taun We | 213 |
| Rugor Nass | 206 |
| Ki-Adi-Mundi | 198 |
| Jar Jar Binks | 196 |
| Mas Amedda | 196 |
| Wat Tambor | 193 |
| Bail Prestor Organa | 191 |
| San Hill | 191 |
| Raymus Antilles | 188 |
| Mace Windu | 188 |
| Plo Koon | 188 |
| Gregar Typho | 185 |
| Adi Gallia | 184 |
| Ric Olié | 183 |
| Boba Fett | 183 |
| Quarsh Panaka | 183 |
| Obi-Wan Kenobi | 182 |
| Jek Tono Porkins | 180 |
| Ackbar | 180 |
| Han Solo | 180 |
| Shaak Ti | 178 |
| Lobot | 175 |
| Darth Maul | 175 |
| Jabba Desilijic Tiure | 175 |
| Wedge Antilles | 170 |
| Palpatine | 170 |
| Finis Valorum | 170 |
| Jocasta Nu | 167 |
| Padmé Amidala | 165 |
| Dormé | 165 |
| Cordé | 157 |
| Mon Mothma | 150 |
| Leia Organa | 150 |
| Watto | 137 |
| R2-D2 | 96 |
| Wicket Systri Warrick | 88 |



In [15]:
text_to_sql(query_engine, "What are the names and birth years of all people from planets with a population greater than 1,000,000?", number=7)


### Question

**7. What are the names and birth years of all people from planets with a population greater than 1,000,000?**

### Answer

Based on the SQL query and results, here is a synthesized response:

The query retrieves the names and birth years of all people from planets with a population greater than 1,000,000. Some notable individuals included are:

- Chewbacca, born 200BBY (Before the Battle of Yavin)
- Jabba the Hutt, born 600BBY 
- Jar Jar Binks, born 52BBY
- Han Solo, born 29BBY
- Leia Organa, born 19BBY
- Palpatine, born 82BBY
- R2-D2, born 33BBY
- Wicket Systri Warrick (an Ewok), born 8BBY
- Wilhuff Tarkin, born 64BBY

The results span a wide range of characters from the Star Wars universe, including humans, droids, Wookiees, Hutts, and Gungans. Many have birth years listed, while others have null values for their birth year. The ordering is alphabetical by name.

### Generated SQL Query
```sql
SELECT p.name, p.birth_year
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE pl.population > 1000000
ORDER BY p.name;
```

### SQL Results

| name | birth_year |
| --- | --- |
| Ackbar | 41BBY |
| Adi Gallia | None |
| Ayla Secura | 48BBY |
| Bail Prestor Organa | 67BBY |
| Bib Fortuna | None |
| Boba Fett | 31.5BBY |
| Bossk | 53BBY |
| Chewbacca | 200BBY |
| Cordé | None |
| Dexter Jettster | None |
| Dormé | None |
| Dud Bolt | None |
| Finis Valorum | 91BBY |
| Greedo | 44BBY |
| Gregar Typho | None |
| Grievous | None |
| Han Solo | 29BBY |
| Jabba Desilijic Tiure | 600BBY |
| Jar Jar Binks | 52BBY |
| Jek Tono Porkins | None |
| Jocasta Nu | None |
| Ki-Adi-Mundi | 92BBY |
| Kit Fisto | None |
| Lama Su | None |
| Lando Calrissian | 31BBY |
| Leia Organa | 19BBY |
| Lobot | 37BBY |
| Mas Amedda | None |
| Mon Mothma | 48BBY |
| Nien Nunb | None |
| Nute Gunray | None |
| Padmé Amidala | 46BBY |
| Palpatine | 82BBY |
| Poggle the Lesser | None |
| Quarsh Panaka | 62BBY |
| R2-D2 | 33BBY |
| Raymus Antilles | None |
| Ric Olié | None |
| Roos Tarpals | None |
| Rugor Nass | None |
| San Hill | None |
| Sebulba | None |
| Tarfful | None |
| Taun We | None |
| Tion Medon | None |
| Wat Tambor | None |
| Watto | None |
| Wedge Antilles | 21BBY |
| Wicket Systri Warrick | 8BBY |
| Wilhuff Tarkin | 64BBY |



In [16]:
text_to_sql(query_engine, "What is the average rotation period of all planets?", number=8)


### Question

**8. What is the average rotation period of all planets?**

### Answer

The average rotation period of all planets is approximately 26 days.

### Generated SQL Query
```sql
SELECT AVG(rotation_period) AS average_rotation_period
FROM planet;
```

### SQL Results

| average_rotation_period |
| --- |
| 25.9583333333333333 |



In [17]:
text_to_sql(query_engine, "List the names, birthyear, and URLs of all people born before the year 20BBY.", number=9)


### Question

**9. List the names, birthyear, and URLs of all people born before the year 20BBY.**

### Answer

Here is a response synthesized from the query results:

The people born before the year 20BBY include Dooku (born 102BBY), C-3PO (born 112BBY), IG-88 (born 15BBY), Luke Skywalker (born 19BBY), Leia Organa (born 19BBY), and Chewbacca (born 200BBY). Dooku and C-3PO were born over a century before the Battle of Yavin, while IG-88, Luke, and Leia were born in the decades just prior. Chewbacca, a wookiee from the planet Kashyyyk, was born an astonishing 200 years before the events of the original Star Wars trilogy. You can find more information about each of these characters at the URLs provided in the query results.

### Generated SQL Query
```sql
SELECT name, birth_year, url 
          FROM people
          WHERE birth_year < '20BBY'
          ORDER BY birth_year;
```

### SQL Results

| name | birth_year | url |
| --- | --- | --- |
| Dooku | 102BBY | https://swapi.co/api/people/67/ |
| C-3PO | 112BBY | https://swapi.co/api/people/2/ |
| IG-88 | 15BBY | https://swapi.co/api/people/23/ |
| Luke Skywalker | 19BBY | https://swapi.co/api/people/1/ |
| Leia Organa | 19BBY | https://swapi.co/api/people/5/ |
| Chewbacca | 200BBY | https://swapi.co/api/people/13/ |



In [18]:
text_to_sql(query_engine, "List the names of all female people and their planet's name.", number=10)


### Question

**10. List the names of all female people and their planet's name.**

### Answer

Based on the SQL query and results, here is a possible response:

The query lists the names of all female people and the names of their home planets. Some notable entries include:

- Leia Organa from the planet Alderaan
- Padmé Amidala from the planet Naboo  
- Rey from an unknown planet
- Shaak Ti from the planet Shili
- Shmi Skywalker from the planet Tatooine
- Jocasta Nu and Adi Gallia from the planet Coruscant
- Luminara Unduli and Barriss Offee from the planet Mirial
- Ayla Secura from the planet Ryloth

The list covers a range of female characters from planets across the Star Wars galaxy, including some very prominent figures as well as lesser-known ones. Several have "Unknown" listed for their planet, likely indicating the planet is not identified in the data.

### Generated SQL Query
```sql
SELECT p.name AS person_name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |

